In [15]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import cv2
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from data_load import PhoneNumberDataset
from data_load import Normalize, Gray_scale_Image, ToTensor, Gray_scale_to_Binary, BoundingBoxes

data_transform = transforms.Compose([Gray_scale_Image(), BoundingBoxes(), ToTensor()])

assert(data_transform is not None)

In [16]:
cross_valid_df = pd.read_csv("data\cross_valid_df.csv")

In [17]:
cross_valid_df.head()

,image_id,phone_number
0,phoneNumberImage22542.png,2599025368
1,phoneNumberImage17984.png,3212222661
2,phoneNumberImage1575.png,8523749904
3,phoneNumberImage30823.png,1931705751
4,phoneNumberImage39894.png,350417540


In [18]:
cross_valid_dataset = PhoneNumberDataset("data\cross_valid_df.csv", "data\phone_number_images", transform=data_transform)

In [19]:
batch_size = 10
cross_valid_dataloader = DataLoader(cross_valid_dataset,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    num_workers=0)

In [20]:
net = torch.load("data\models\model_basic\modelWith2EpochOperaAppliedOnEachDigitBatchSize320.pth")
net.eval()

Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (mpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=80, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=80, out_features=10, bias=True)
)

In [21]:
def net_sample_output():
    for i, sample in enumerate(cross_valid_dataloader):
        images = sample['image']
        phone_num_digits_list = sample['digits_list']
        images = images.type(torch.FloatTensor)
        output_digits = net(images)
        print(output_digits[i][1])
        output_digits = torch.argmax(output_digits, dim=1)
        output_digits = output_digits.view(output_digits.size()[0], -1)
        
        if i == 0:
            return images, output_digits, phone_num_digits_list

In [22]:
def show_Image(image):
    plt.imshow(image, cmap='gray')

test_images, test_outputs, ground_truth_output = net_sample_output()
print(test_images.data.size())
print(test_outputs.data.size())
print(len(ground_truth_output))

tensor(0., grad_fn=<SelectBackward0>)
torch.Size([10, 10, 28, 28])
torch.Size([100, 1])
10


In [23]:
def show_Image(image):
    plt.imshow(image, cmap='gray')

def visualize_output(test_images, test_outputs, ground_truth_output, batch_size=4):
    for i in range(batch_size):

        image = test_images[i].data
        image = image.numpy()

        test_digits = test_outputs[i*10:i*10 + 10].data.reshape((10))
        test_digits = test_digits.numpy()
        
        print("test digits:   ", test_digits)
        
        grond_digits = [x.data.numpy() for x in ground_truth_output]
        grond_digits = np.asarray(grond_digits).T
        grond_digits = grond_digits[i]

        print("ground digits: ", grond_digits)
        print("\n")

        digits_list = [x.data.numpy() for x in ground_truth_output]
        digits_batch = torch.Tensor(np.transpose(np.asarray(digits_list)))
        digits_one_hot = []
        for phone_num in digits_batch:
            one_hot_encoded_digit = F.one_hot(phone_num.type(torch.LongTensor), num_classes=10)
            digits_one_hot.append(one_hot_encoded_digit)
        digits_one_hot_tensor = [x.data.numpy() for x in digits_one_hot]
        digits_one_hot_tensor = np.array(digits_one_hot_tensor)
        digits_one_hot_tensor = torch.Tensor(digits_one_hot_tensor)


In [24]:
visualize_output(test_images, test_outputs, ground_truth_output=ground_truth_output)

test digits:    [3 1 7 4 9 6 6 1 1 9]
ground digits:  [3 1 7 4 9 6 6 1 1 8]


test digits:    [8 8 0 2 0 3 8 6 4 8]
ground digits:  [8 8 0 2 0 3 8 6 4 8]


test digits:    [1 8 9 3 4 2 3 2 1 1]
ground digits:  [1 8 9 3 4 2 3 2 1 1]


test digits:    [4 9 7 8 6 5 8 7 4 7]
ground digits:  [4 9 2 8 6 5 8 7 4 7]




## Results on Cross validation data

In [25]:
def cross_valid_net():
    
    net.eval()
    running_accu = 0.0

    for batch_i, data in enumerate(cross_valid_dataloader):

        images = data['image']
        digits_list = data['digits_list']

        digits_list = [x.data.numpy() for x in digits_list]
        digits_batch = torch.Tensor(np.transpose(np.asarray(digits_list)))
        # print(digits_batch)
        digits_batch = torch.reshape(digits_batch,(digits_batch.size(0)*digits_batch.size(1), ))
        # print(digits_batch.size())
        digits_one_hot = []
        for phone_num in digits_batch:
            one_hot_encoded_digit = F.one_hot(phone_num.type(torch.LongTensor), num_classes=10)
            digits_one_hot.append(one_hot_encoded_digit)
        digits_one_hot_tensor = [x.data.numpy() for x in digits_one_hot]
        digits_one_hot_tensor = np.array(digits_one_hot_tensor)
        digits_one_hot_tensor = torch.Tensor(digits_one_hot_tensor)
        images = images.type(torch.FloatTensor)

        output_digits = net(images)
        
        output_digits = output_digits.type(torch.FloatTensor)
        #Accuracy
        out = torch.argmax(output_digits, dim=1)            
        # print(out.size())
        accuracy = out == digits_batch
        accuracy = torch.sum(accuracy.detach()).float()

        running_accu += accuracy

        if(batch_i % 10 == 9):
            print(f'Batch: {batch_i + 1}, Avg. Accu: {running_accu}')
            running_accu = 0.0
            
    print("Cross Validation is finished")

In [26]:
cross_valid_net()# total images are 1000

Batch: 10, Avg. Accu: 966.0
Batch: 20, Avg. Accu: 955.0
Batch: 30, Avg. Accu: 952.0
Batch: 40, Avg. Accu: 956.0
Batch: 50, Avg. Accu: 965.0
Batch: 60, Avg. Accu: 962.0
Batch: 70, Avg. Accu: 956.0
Batch: 80, Avg. Accu: 946.0
Batch: 90, Avg. Accu: 944.0
Batch: 100, Avg. Accu: 947.0
Batch: 110, Avg. Accu: 957.0
Batch: 120, Avg. Accu: 962.0
Batch: 130, Avg. Accu: 966.0
Batch: 140, Avg. Accu: 938.0
Batch: 150, Avg. Accu: 950.0
Batch: 160, Avg. Accu: 951.0
Batch: 170, Avg. Accu: 959.0
Batch: 180, Avg. Accu: 960.0
Batch: 190, Avg. Accu: 965.0
Batch: 200, Avg. Accu: 953.0
Batch: 210, Avg. Accu: 945.0
Batch: 220, Avg. Accu: 953.0
Batch: 230, Avg. Accu: 959.0
Batch: 240, Avg. Accu: 954.0
Batch: 250, Avg. Accu: 954.0
Batch: 260, Avg. Accu: 962.0
Batch: 270, Avg. Accu: 950.0
Batch: 280, Avg. Accu: 948.0
Batch: 290, Avg. Accu: 949.0
Batch: 300, Avg. Accu: 961.0
Batch: 310, Avg. Accu: 957.0
Batch: 320, Avg. Accu: 956.0
Batch: 330, Avg. Accu: 962.0
Batch: 340, Avg. Accu: 957.0
Batch: 350, Avg. Accu: 